## Pulling the repo

In [5]:
import os
import sys

# --- CONFIGURATION ---
REPO_NAME = "FusionKAN"
# Tip: If private, use: https://<your_token>@github.com/yourname/FusionKAN.git
REPO_URL = "https://github.com/DonKamilo00/FusionKAN.git" 
BRANCH = "main" 

# --- SETUP LOGIC ---
if not os.path.exists(REPO_NAME):
    print(f"🚀 Cloning {REPO_NAME}...")
    !git clone {REPO_URL}
    %cd {REPO_NAME}
else:
    print(f"🔄 Repo exists. Updating...")
    %cd {REPO_NAME}
    !git fetch origin
    !git reset --hard origin/{BRANCH} # Force overwrite local changes to match remote

# --- INSTALL BUILD DEPENDENCIES ---
# Ninja makes C++ compilation much faster
!pip install ninja 

# --- COMPILE & INSTALL FUSIONKAN ---
print("⚙️ Compiling CUDA Kernels (this may take a moment)...")
# --no-deps: Don't waste time checking torch/numpy installation every time
# --force-reinstall: Ensures the C++ extension is actually rebuilt
!pip install . --verbose --no-deps --force-reinstall

print("✅ Setup Complete. FusionKAN is ready.")

🚀 Cloning FusionKAN...
Cloning into 'FusionKAN'...
remote: Enumerating objects: 223, done.
remote: Counting objects: 100% (223/223), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 223 (delta 119), reused 166 (delta 63), pack-reused 0 (from 0)
Receiving objects: 100% (223/223), 201.71 KiB | 25.21 MiB/s, done.
Resolving deltas: 100% (119/119), done.
/content/FusionKAN/FusionKAN
⚙️ Compiling CUDA Kernels (this may take a moment)...
Using pip 24.1.2 from /usr/local/lib/python3.12/dist-packages/pip (python 3.12)
Processing /content/FusionKAN/FusionKAN
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-2qmh005m/fusion_kan.egg-info
  writing /tmp/pip-pip-egg-info-2qmh005m/fusion_kan.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-2qmh005m/fusion_kan.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-2qmh005m/fusion_kan.egg-info/requires.txt
  writing top-level names to /tmp/pip

✅ Setup Complete. FusionKAN is ready.


# Saving changes to the .Cu

In [ ]:
git add .
git commit -m "update kernel"
git push

# Updating

In [ ]:
# Run this cell whenever you push changes to GitHub
%cd /content/FusionKAN
!git pull
!pip install . --no-deps --force-reinstall
print("✅ Library Updated")

# Verification

In [6]:
import torch
import fusion_kan
from fusion_kan import FusionKANLayer

# Verify the C++ backend loaded correctly
try:
    from fusion_kan.functional import _backend
    print("CUDA Backend Loaded Successfully")
except ImportError:
    print("❌ Error: CUDA Backend not found. Did compilation fail?")

# --- Your Benchmark / Test Code Here ---
batch_size = 4096
in_features = 32
out_features = 64

layer = FusionKANLayer(in_features, out_features).cuda()
x = torch.randn(batch_size, in_features).cuda()

# Forward pass
y = layer(x)
print(f"Forward pass output shape: {y.shape}")

# Backward pass (Critical for testing your new kernel gradients)
loss = y.sum()
loss.backward()
print("Backward pass successful")

CUDA Backend Loaded Successfully
Forward pass output shape: torch.Size([4096, 64])
Backward pass successful
